<h2/> Import all the necessary libraries. </h2>

In [189]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from math import cos, acos

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library
import itertools
import matplotlib.cm as cm
import matplotlib.colors as colors
from folium.features import DivIcon

<h3/>Center of Montréal island, which is the region under scope.</h3>

In [101]:
lat_lt = 45.545773
long_lt = -73.686490  #hor
lat_rt = 45.545773
long_rt = -73.550903  #hor
lat_lb = 45.445473
long_lb = -73.686490  #hor
lat_rb = 45.445473
long_rb = -73.550903  #hor

I want to find a good radius to use in foursquare api. <br>
Convert the change in latitude and longitude values into meters.

In [199]:
def lat_to_m(lat1, lat2):
    return abs(lat2 - lat1)* 111.32 * 1000
     
def long_to_m(long1, long2, lat):
    return abs(long2 - long1) * 40075 * cos(lat/360*2*np.pi)/360 * 1000 

def m_to_lat(m):
    return m/(111.32 * 1000)
def m_to_long(m, lat):
    return 360 * m / (40075 * 1000 * cos(lat/360*2*np.pi))

In [102]:
vert = lat_to_m(lat_lb, lat_lt)
hor = long_to_m(long_lt, long_rt, lat_lt)

Calculate the dimension of the region of interest.

In [201]:
print('The width of the region is {}m, and the height of the region is {}m.'.format(hor, vert))

The width of the region is 10570.548852935093m, and the height of the region is 11165.395999999684m.


In [ ]:
# radius of each center is width divided into 5.
radius = hor/5

For this script, I will be dividing the region into 16 parts and study them individually. <br>
<b/>You can see the divided parts in the first graph at the bottom of this script. </b> <br>
So, let's calculate the coordinates of the each center and put them into a dataframe.

In [106]:
latlist = list(np.linspace(start = lat_lb, stop = lat_lt, num = 6) )[1:5]
longlist = list(np.linspace(start = long_lt, stop = long_rt, num = 6))[1:5]

In [115]:
points = list(range(1,17))
t = list(itertools.product(latlist, longlist))
df = pd.DataFrame(t, columns = ('latitude', 'longitude'))
df['points']= points
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]
df


,points,latitude,longitude
0,1,45.465533,-73.659373
1,2,45.465533,-73.632255
2,3,45.465533,-73.605138
3,4,45.465533,-73.578020
4,5,45.485593,-73.659373
5,6,45.485593,-73.632255
6,7,45.485593,-73.605138
7,8,45.485593,-73.578020
8,9,45.505653,-73.659373
9,10,45.505653,-73.632255


In [116]:
CLIENT_ID = '' # This is hidden
CLIENT_SECRET = '' # This is hidden.
VERSION = '20180604'
LIMIT = 100

In [117]:
# for all the neighbourhoods
def getNearbyVenues(names, latitudes, longitudes, radius):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['points', 
                  'points Latitude', 
                  'points Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [119]:
# type your answer here

montreal_venues = getNearbyVenues(names=df['points'],
                                   latitudes=df['latitude'],
                                   longitudes=df['longitude'],
                                   radius=radius
                                  )
print(montreal_venues.shape)
montreal_venues.head()



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
(1427, 7)


,points,points Latitude,points Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,1,45.465533,-73.659373,Cote Saint-Luc Aquatic and Community Centre,45.468675,-73.670348,Pool
1,1,45.465533,-73.659373,IGA,45.476430,-73.666752,Grocery Store
2,1,45.465533,-73.659373,West End Gym V2,45.479114,-73.666677,Gym
3,1,45.465533,-73.659373,Express Indien,45.467041,-73.638893,Indian Restaurant
4,1,45.465533,-73.659373,Burger de Ville,45.453237,-73.646031,Burger Joint


Foursquares uses various subcategories rather than the parent category 'Nightlife Spot'. <br>
Need to change all the subcategrories into Nightlife spot.

In [128]:
montreal_venues['Venue Category'] = montreal_venues['Venue Category'].replace(['Beach Bar',
'Beer Bar',
'Beer Garden',
'Champagne Bar',
'Cocktail Bar',
'Dive Bar',
'Gay Bar',
'Hookah Bar',
'Hotel Bar',
'Karaoke Bar',
'Pub',
'Sake Bar',
'Speakeasy',
'Sports Bar',
'Tiki Bar',
'Whisky Bar',
'Wine Bar',
'Brewery',
'Lounge',
'Night Market',
'Nightclub',
'Other Nightlife',
'Strip Club'], 'Nightlife Spot'
                            )


In [129]:
montreal_venues.groupby('points').count()

,points Latitude,points Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
points,,,,,,
1,42,42,42,42,42,42
2,100,100,100,100,100,100
3,100,100,100,100,100,100
4,100,100,100,100,100,100
5,67,67,67,67,67,67
6,100,100,100,100,100,100
7,100,100,100,100,100,100
8,100,100,100,100,100,100
9,100,100,100,100,100,100


In [130]:
print('There are {} uniques categories.'.format(len(montreal_venues['Venue Category'].unique())))

There are 185 uniques categories.


Let's use one hot encoding to calculate the frequency.

In [131]:
# one hot encoding
montreal_onehot = pd.get_dummies(montreal_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
montreal_onehot['points'] = montreal_venues['points'] 

# move neighborhood column to the first column
fixed_columns = [montreal_onehot.columns[-1]] + list(montreal_onehot.columns[:-1])
montreal_onehot = montreal_onehot[fixed_columns]

montreal_onehot.head()

,points,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Women's Store,Yoga Studio
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group by the points of interest.

In [132]:
montreal_grouped = montreal_onehot.groupby('points').mean().reset_index()
montreal_grouped

,points,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Women's Store,Yoga Studio
0,1,0.00,0.02381,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,...,0.00,0.000000,0.00,0.023810,0.02381,0.00,0.000000,0.000000,0.000000,0.00
1,2,0.00,0.01000,0.000000,0.00,0.000000,0.00,0.010000,0.00,0.000000,...,0.00,0.000000,0.00,0.010000,0.00000,0.02,0.010000,0.000000,0.000000,0.01
2,3,0.00,0.00000,0.000000,0.00,0.010000,0.01,0.010000,0.00,0.000000,...,0.00,0.000000,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.01
3,4,0.00,0.00000,0.000000,0.00,0.000000,0.03,0.000000,0.00,0.000000,...,0.00,0.000000,0.01,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.01
4,5,0.00,0.00000,0.014925,0.00,0.000000,0.00,0.014925,0.00,0.014925,...,0.00,0.014925,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.00
5,6,0.00,0.00000,0.010000,0.00,0.000000,0.00,0.000000,0.00,0.000000,...,0.00,0.010000,0.00,0.000000,0.00000,0.01,0.040000,0.000000,0.000000,0.01
6,7,0.00,0.00000,0.010000,0.00,0.000000,0.01,0.000000,0.00,0.000000,...,0.00,0.000000,0.00,0.000000,0.00000,0.01,0.000000,0.000000,0.000000,0.02
7,8,0.00,0.00000,0.010000,0.01,0.000000,0.00,0.000000,0.00,0.000000,...,0.00,0.000000,0.00,0.000000,0.00000,0.01,0.000000,0.000000,0.000000,0.00
8,9,0.00,0.01000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.010000,...,0.00,0.010000,0.00,0.010000,0.00000,0.00,0.020000,0.000000,0.000000,0.00
9,10,0.00,0.00000,0.024390,0.00,0.000000,0.00,0.000000,0.00,0.000000,...,0.00,0.000000,0.00,0.000000,0.00000,0.00,0.048780,0.000000,0.000000,0.00


Find the top 5 most frequent venues.

In [134]:
num_top_venues = 5

for hood in montreal_grouped['points']:
    print("----"+str(hood)+"----")
    temp = montreal_grouped[montreal_grouped['points'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----1----
            venue  freq
0        Pharmacy  0.10
1  Ice Cream Shop  0.07
2   Grocery Store  0.05
3    Skating Rink  0.05
4   Shopping Mall  0.05


----2----
                  venue  freq
0           Coffee Shop  0.06
1  Fast Food Restaurant  0.05
2                  Café  0.05
3              Pharmacy  0.05
4     Indian Restaurant  0.04


----3----
            venue  freq
0     Coffee Shop  0.05
1  Nightlife Spot  0.05
2            Café  0.05
3      Restaurant  0.04
4            Park  0.04


----4----
            venue  freq
0            Café  0.08
1  Nightlife Spot  0.08
2  Breakfast Spot  0.06
3      Restaurant  0.06
4          Bakery  0.05


----5----
                  venue  freq
0  Fast Food Restaurant  0.07
1            Restaurant  0.06
2           Coffee Shop  0.06
3                   Gym  0.06
4              Pharmacy  0.04


----6----
                   venue  freq
0            Coffee Shop  0.08
1                   Park  0.06
2               Pharmacy  0.04
3             

In [135]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [137]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['points']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['points'] = montreal_grouped['points']

for ind in np.arange(montreal_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(montreal_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,points,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,1,Pharmacy,Ice Cream Shop,Skating Rink,Shopping Mall,Grocery Store
1,2,Coffee Shop,Café,Pharmacy,Fast Food Restaurant,Korean Restaurant
2,3,Nightlife Spot,Café,Coffee Shop,Park,Restaurant
3,4,Café,Nightlife Spot,Restaurant,Breakfast Spot,Bakery
4,5,Fast Food Restaurant,Gym,Restaurant,Coffee Shop,Indian Restaurant
5,6,Coffee Shop,Park,Café,Pharmacy,Vietnamese Restaurant
6,7,Café,Bakery,Park,Breakfast Spot,French Restaurant
7,8,Café,Restaurant,Japanese Restaurant,French Restaurant,Coffee Shop
8,9,Fast Food Restaurant,Pharmacy,Coffee Shop,Pizza Place,Chinese Restaurant
9,10,Coffee Shop,Pharmacy,Bakery,Park,Bar


Let's find points(regions) that have Nightlife spot in the top 5 frequent venues.

In [168]:
rec_df = neighborhoods_venues_sorted[neighborhoods_venues_sorted.isin(["Nightlife Spot"]).any(axis=1)]
rec_df.head()

,points,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,3,Nightlife Spot,Café,Coffee Shop,Park,Restaurant
3,4,Café,Nightlife Spot,Restaurant,Breakfast Spot,Bakery
14,15,Café,Restaurant,Nightlife Spot,Pizza Place,Breakfast Spot
15,16,Café,French Restaurant,Bakery,Nightlife Spot,Park


Make a list of these points and merge the latitude and longitude data from the original dataframe.

In [172]:
rec_list = list(neighborhoods_venues_sorted[neighborhoods_venues_sorted.isin(["Nightlife Spot"]).any(axis=1)]['points'])
n_reg=len(rec_list)
rec_list

[3, 4, 15, 16]

In [180]:
rec_merged = df[df[['points']].isin(rec_list).any(axis=1)]
rec_merged

,points,latitude,longitude
2,3,45.465533,-73.605138
3,4,45.465533,-73.578020
14,15,45.525713,-73.605138
15,16,45.525713,-73.578020


In [203]:
# Calculate the latitude and longitude of the center of the region of interest (center of montreal island)
latitude = (lat_lt + lat_lb)/2
longitude = (long_lt + long_rt)/2

In [1]:
montreal_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
montreal_merged = montreal_merged.join(neighborhoods_venues_sorted.set_index('points'), on='points')

montreal_merged # check the last columns!

NameError: name 'df' is not defined

In [197]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(n_reg)
ys = [i + x + (i*x)**2 for i in range(n_reg)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi in zip(montreal_merged['latitude'], montreal_merged['longitude'], montreal_merged['points']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.Circle(
        [lat, lon],
        radius=radius,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
    folium.map.Marker(
    [lat, lon],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12pt">%s</div>' % str(poi),
        )).add_to(map_clusters)
       
map_clusters

In [204]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(n_reg)
ys = [i + x + (i*x)**2 for i in range(n_reg)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi in zip(rec_merged['latitude'], rec_merged['longitude'], rec_merged['points']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.Circle(
        [lat, lon],
        radius=radius,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
    folium.map.Marker(
    [lat, lon],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12pt">%s</div>' % str(poi),
        )).add_to(map_clusters)
       
map_clusters

<h2> Recommended regions are 3,4,15 and 16, with radius of about 2km. </h2>